# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma

# Auxiliary functions

In [ ]:
def gaussian(x, mean, std):
    return (1/(np.sqrt(2*np.pi)*std))*np.exp(-0.5*((x-mean)/std)**2)

In [ ]:
def calculate_bin(data_column):
    return int(1 + 3.3 * np.log10(len(data_column)))

In [ ]:
def gamma_function(shape):
    gamma = [1]
    for i in range(1, int(shape)+1):
        gamma.append(gamma[i-1] * i)
    return gamma[-2]

In [ ]:
def gamma_distribution(x, shape, scale):
    return (x**(shape-1) * np.exp(-x/scale)) / (scale**shape * gamma_function(shape))

# Read Data

In [ ]:
df_chromecast = pd.read_csv("../data/dataset_chromecast.csv")
df_smart_tv = pd.read_csv("../data/dataset_smart-tv.csv")

# Data Preprocessing

## Log Transformation

### Chromecast

In [ ]:
df_chromecast["log_bytes_down"] = np.log10(df_smart_tv["bytes_down"] + 1)
df_chromecast["log_bytes_up"] = np.log10(df_smart_tv["bytes_up"] + 1)

### Smart TV

In [ ]:
df_smart_tv["log_bytes_down"] = np.log10(df_smart_tv["bytes_down"] + 1)
df_smart_tv["log_bytes_up"] = np.log10(df_smart_tv["bytes_up"] + 1)

## Create Hour Column

### Chromecast

In [ ]:
df_chromecast["hour"] = df_chromecast["date_hour"].apply(
    lambda x: int(x.split(" ")[1].split(":")[0])
)

### Smart TV

In [ ]:
df_smart_tv["hour"] = df_smart_tv["date_hour"].apply(
    lambda x: int(x.split(" ")[1].split(":")[0])
)

# Second Section

## Chromecast

### Histogram

In [ ]:
chromecast_hist_bin = calculate_bin(df_chromecast["log_bytes_up"])
fig = plt.figure()
plt.hist(
    df_chromecast["log_bytes_up"],
    bins=chromecast_hist_bin,
    color="blue",
    alpha=0.5,
    edgecolor="black",
)
plt.xlabel("Bytes Up")
plt.ylabel("Frequency")
plt.title("Chromecast histogram of Bytes Up")
fig.savefig(
    "../images/chromecast/general/histogram/chromecast_hist_bytes_up.png", 
    dpi=fig.dpi
)

In [ ]:
chromecast_hist_bin = calculate_bin(df_chromecast["log_bytes_down"])
fig = plt.figure()
plt.hist(
    df_chromecast["log_bytes_down"],
    bins=chromecast_hist_bin,
    color="red",
    alpha=0.5,
    edgecolor="black",
)
plt.xlabel("Bytes Down")
plt.ylabel("Frequency")
plt.title("Chromecast histogram of Bytes Down")
fig.savefig(
    "../images/chromecast/general/histogram/chromecast_hist_bytes_down.png", 
    dpi=fig.dpi
)

### Empirical Distribution Function

In [ ]:
fig = plt.figure()
plt.plot(
    df_chromecast["log_bytes_up"].sort_values(),
    np.linspace(0, 1, len(df_chromecast["log_bytes_up"])),
    color="blue",
    marker=".",
)
plt.xlabel("Bytes Up")
plt.ylabel("Fx(x)")
plt.title("Chromecast Empirical Distribution Function of Bytes Up")
fig.savefig(
    "../images/chromecast/general/edf/chromecast_edf_bytes_up.png", 
    dpi=fig.dpi
)

In [ ]:
fig = plt.figure()
plt.plot(
    df_chromecast["log_bytes_down"].sort_values(),
    np.linspace(0, 1, len(df_chromecast["log_bytes_down"])),
    color="red",
    marker=".",
)
plt.xlabel("Bytes Down")
plt.ylabel("Fx(x)")
plt.title("Chromecast Empirical Distribution Function of Bytes Down")
fig.savefig(
    "../images/chromecast/general/edf/chromecast_edf_bytes_down.png", 
    dpi=fig.dpi
)

### Boxplot

In [ ]:
fig = plt.figure()
plt.boxplot(
    [df_chromecast["log_bytes_up"], df_chromecast["log_bytes_down"]],
    labels=["Bytes Up", "Bytes Down"],
    patch_artist=True,
    boxprops=dict(facecolor="cyan", color="black"),
    medianprops=dict(color="black"),
    whiskerprops=dict(color="black"),
    capprops=dict(color="black"),
)
plt.ylabel("Bytes")
plt.title("Chromecast Boxplot of Bytes Up and Bytes Down")
fig.savefig(
    "../images/chromecast/general/boxplot/chromecast_boxplot_bytes_up_down.png",
    dpi=fig.dpi,
)

### Statistical Analysis

In [ ]:
chromecast_bytes_up_down = pd.DataFrame(
    {
        "Bytes Up": [
            df_chromecast["log_bytes_up"].mean(),
            df_chromecast["log_bytes_up"].var(),
            df_chromecast["log_bytes_up"].std(),
        ],
        "Bytes Down": [
            df_chromecast["log_bytes_down"].mean(),
            df_chromecast["log_bytes_down"].var(),
            df_chromecast["log_bytes_down"].std(),
        ],
    },
    index=["Mean", "Variance", "Standard Deviation"],
)

chromecast_bytes_up_down

## Smart TV

### Histogram

In [ ]:
smart_tv_hist_bin = calculate_bin(df_smart_tv["log_bytes_up"])
fig = plt.figure()
plt.hist(
    df_smart_tv["log_bytes_up"],
    bins=smart_tv_hist_bin,
    color="blue",
    alpha=0.5,
    edgecolor="black",
)
plt.xlabel("Bytes Up")
plt.ylabel("Frequency")
plt.title("Smart TV histogram of Bytes Up")
fig.savefig(
    "../images/smart_tv/general/histogram/smart_tv_hist_bytes_up.png", 
    dpi=fig.dpi
)

In [ ]:
smart_tv_hist_bin = calculate_bin(df_smart_tv["log_bytes_down"])
fig = plt.figure()
plt.hist(
    df_smart_tv["log_bytes_down"],
    bins=smart_tv_hist_bin,
    color="red",
    alpha=0.5,
    edgecolor="black",
)
plt.xlabel("Bytes Down")
plt.ylabel("Frequency")
plt.title("Smart TV histogram of Bytes Down")
fig.savefig(
    "../images/smart_tv/general/histogram/smart_tv_hist_bytes_down.png", 
    dpi=fig.dpi
)

### Empirical Distribution Function

In [ ]:
fig = plt.figure()
plt.plot(
    df_smart_tv["log_bytes_up"].sort_values(),
    np.linspace(0, 1, len(df_smart_tv["log_bytes_up"])),
    color="blue",
    marker=".",
)
plt.xlabel("Bytes Up")
plt.ylabel("Fx(x)")
plt.title("Smart TV Empirical Distribution Function of Bytes Up")
fig.savefig(
    "../images/smart_tv/general/edf/smart_tv_edf_bytes_up.png", 
    dpi=fig.dpi
)

In [ ]:
fig = plt.figure()
plt.plot(
    df_smart_tv["log_bytes_down"].sort_values(),
    np.linspace(0, 1, len(df_smart_tv["log_bytes_down"])),
    color="red",
    marker=".",
)
plt.xlabel("Bytes Down")
plt.ylabel("Fx(x)")
plt.title("Smart TV Empirical Distribution Function of Bytes Down")
fig.savefig(
    "../images/smart_tv/general/edf/smart_tv_edf_bytes_down.png", 
    dpi=fig.dpi
)

### Boxplot

In [ ]:
fig = plt.figure()
plt.boxplot(
    [df_smart_tv["log_bytes_up"], df_smart_tv["log_bytes_down"]],
    labels=["Bytes Up", "Bytes Down"],
    patch_artist=True,
    boxprops=dict(facecolor="cyan", color="black"),
    medianprops=dict(color="black"),
    whiskerprops=dict(color="black"),
    capprops=dict(color="black"),
)
plt.ylabel("Bytes")
plt.title("Smart TV Boxplot of Bytes Up and Bytes Down")
fig.savefig(
    "../images/smart_tv/general/boxplot/smart_tv_boxplot_bytes_up_down.png", 
    dpi=fig.dpi
)

### Statistical Analysis

In [ ]:
smart_tv_bytes_up_down = pd.DataFrame(
    {
        "Bytes Up": [
            df_smart_tv["log_bytes_up"].mean(),
            df_smart_tv["log_bytes_up"].var(),
            df_smart_tv["log_bytes_up"].std(),
        ],
        "Bytes Down": [
            df_smart_tv["log_bytes_down"].mean(),
            df_smart_tv["log_bytes_down"].var(),
            df_smart_tv["log_bytes_down"].std(),
        ],
    },
    index=["Mean", "Variance", "Standard Deviation"],
)
smart_tv_bytes_up_down

# Third Section

## Chromecast

### Boxplot

In [ ]:
for hour in range(0, 24):
    fig = plt.figure()
    plt.boxplot(
        [
            df_chromecast[df_chromecast["hour"] == hour]["log_bytes_up"],
            df_chromecast[df_chromecast["hour"] == hour]["log_bytes_down"],
        ],
        labels=["Bytes Up", "Bytes Down"],
        patch_artist=True,
        boxprops=dict(facecolor="cyan", color="black"),
        medianprops=dict(color="black"),
        whiskerprops=dict(color="black"),
        capprops=dict(color="black"),
    )
    plt.ylabel("Bytes")
    plt.title("Chromecast Boxplot of Bytes Up and Bytes Down for Hour " + str(hour))
    fig.savefig(
        f"../images/chromecast/hourly/boxplot/chromecast_boxplot_bytes_up_down_hour_{hour}.png",
        dpi=fig.dpi,
    )

### Statistical Analysis

In [ ]:
fig = plt.figure()
plt.plot(df_chromecast.groupby("hour")["log_bytes_up"].mean(), color="red", marker=".")
plt.plot(df_chromecast.groupby("hour")["log_bytes_up"].var(), color="green", marker=".")
plt.plot(df_chromecast.groupby("hour")["log_bytes_up"].std(), color="blue", marker=".")
plt.xlabel("Hour")
plt.ylabel("Bytes")
plt.legend(["Mean", "Variance", "Standard Deviation"])
plt.title("Chromecast Bytes Up Mean, Variance and Standard Deviation by Hour")
fig.savefig(
    "../images/chromecast/hourly/statistical_analysis/chromecast_line_bytes_up_mean_var_std_hour.png",
    dpi=fig.dpi,
)

## Smart TV

### Boxplot

In [ ]:
for hour in range(0, 24):
    fig = plt.figure()
    plt.boxplot(
        [
            df_smart_tv[df_smart_tv["hour"] == hour]["log_bytes_up"],
            df_smart_tv[df_smart_tv["hour"] == hour]["log_bytes_down"],
        ],
        labels=["Bytes Up", "Bytes Down"],
        patch_artist=True,
        boxprops=dict(facecolor="cyan", color="black"),
        medianprops=dict(color="black"),
        whiskerprops=dict(color="black"),
        capprops=dict(color="black"),
    )
    plt.ylabel("Bytes")
    plt.title("Smart TV Boxplot of Bytes Up and Bytes Down for Hour " + str(hour))
    fig.savefig(
        f"../images/smart_tv/hourly/boxplot/smart_tv_boxplot_bytes_up_down_hour_{hour}.png",
        dpi=fig.dpi,
    )

### Statistical Analysis

In [ ]:
fig = plt.figure()
plt.plot(df_smart_tv.groupby("hour")["log_bytes_up"].mean(), color="red", marker=".")
plt.plot(df_smart_tv.groupby("hour")["log_bytes_up"].var(), color="green", marker=".")
plt.plot(df_smart_tv.groupby("hour")["log_bytes_up"].std(), color="blue", marker=".")
plt.xlabel("Hour")
plt.ylabel("Bytes")
plt.legend(["Mean", "Variance", "Standard Deviation"])
plt.title("Smart TV Bytes Up Mean, Variance and Standard Deviation by Hour")
fig.savefig(
    "../images/smart_tv/hourly/statistical_analysis/smart_tv_line_bytes_up_mean_var_std_hour.png",
    dpi=fig.dpi,
)

# Fourth Section

## Chromecast

### Filter Dataset

In [ ]:
hour_of_max_median_up_chromecast = df_chromecast.groupby("hour")["bytes_up"].median().idxmax()
hour_of_max_mean_up_chromecast = df_chromecast.groupby("hour")["bytes_up"].mean().idxmax()
hour_of_max_median_down_chromecast = df_chromecast.groupby("hour")["bytes_down"].median().idxmax()
hour_of_max_mean_down_chromecast = df_chromecast.groupby("hour")["bytes_down"].mean().idxmax()

In [ ]:
print(
    f"""
        Hour of max median bytes up: {hour_of_max_median_up_chromecast},
        Hour of max median bytes down: {hour_of_max_median_down_chromecast}, 
        Hour of max mean bytes up: {hour_of_max_mean_up_chromecast}, 
        Hour of max mean bytes down: {hour_of_max_mean_down_chromecast}
    """
    )

In [ ]:
df_chromecast_max_median_up = df_chromecast[
    (df_chromecast["hour"] == hour_of_max_median_up_chromecast)
][["hour", "bytes_up", "log_bytes_up", "device_id"]]

df_chromecast_max_mean_up = df_chromecast[
    (df_chromecast["hour"] == hour_of_max_mean_up_chromecast)
][["hour", "bytes_up", "log_bytes_up", "device_id"]]

df_chromecast_max_median_down = df_chromecast[
    (df_chromecast["hour"] == hour_of_max_median_down_chromecast)
][["hour", "bytes_down", "log_bytes_down", "device_id"]]

df_chromecast_max_mean_down = df_chromecast[
    (df_chromecast["hour"] == hour_of_max_mean_down_chromecast)
][["hour", "bytes_down", "log_bytes_down", "device_id"]]

### Histogram

In [ ]:
for df, column in [
    (df_chromecast_max_median_up, "log_bytes_up"),
    (df_chromecast_max_median_down, "log_bytes_down"),
    (df_chromecast_max_mean_up, "log_bytes_up"),
    (df_chromecast_max_mean_down, "log_bytes_down")
]:
    chromecast_hist_bin = calculate_bin(df[column])
    fig = plt.figure()
    plt.hist(df[column], bins=chromecast_hist_bin, color=("blue" if column=="log_bytes_up" else "red"), alpha=0.5, edgecolor="black")
    plt.xlabel("Bytes")
    plt.ylabel("Frequency")
    plt.title(f"Smart TV Histogram of {column} for Hour " + str(df["hour"].iloc[0]))
    fig.savefig(
        f"../images/chromecast/fourth_section/histogram/chromecast_histogram_{column}_hour_{df['hour'].iloc[0]}.png",
        dpi=fig.dpi,
    )

### MLE

#### Gamma

In [ ]:
chromecast_shape_median_up, _, chromecast_scale_median_up = gamma.fit(df_chromecast_max_median_up["log_bytes_up"])
chromecast_shape_median_down, _, chromecast_scale_median_down = gamma.fit(df_chromecast_max_median_down["log_bytes_down"])
chromecast_shape_mean_up, _, chromecast_scale_mean_up = gamma.fit(df_chromecast_max_mean_up["log_bytes_up"])
chromecast_shape_mean_down, _, chromecast_scale_mean_down = gamma.fit(df_chromecast_max_mean_down["log_bytes_down"])

In [ ]:
chromecast_gamma_median_up = gamma_distribution(df_chromecast_max_median_up["log_bytes_up"], chromecast_shape_median_up, chromecast_scale_median_up)
chromecast_gamma_median_down = gamma_distribution(df_chromecast_max_median_down["log_bytes_down"], chromecast_shape_median_down, chromecast_scale_median_down)
chromecast_gamma_mean_up = gamma_distribution(df_chromecast_max_mean_up["log_bytes_up"], chromecast_shape_mean_up, chromecast_scale_mean_up)
chromecast_gamma_mean_down = gamma_distribution(df_chromecast_max_mean_down["log_bytes_down"], chromecast_shape_mean_down, chromecast_scale_mean_down)

#### Gaussian

In [ ]:
chromecast_max_median_up_mean = df_chromecast_max_median_up["log_bytes_up"].mean()
chromecast_max_median_up_std = df_chromecast_max_median_up["log_bytes_up"].std()

chromecast_max_median_down_mean = df_chromecast_max_median_down["log_bytes_down"].mean()
chromecast_max_median_down_std = df_chromecast_max_median_down["log_bytes_down"].std()

chromecast_max_mean_up_mean = df_chromecast_max_mean_up["log_bytes_up"].mean()
chromecast_max_mean_up_std = df_chromecast_max_mean_up["log_bytes_up"].std()

chromecast_max_mean_down_mean = df_chromecast_max_mean_down["log_bytes_down"].mean()
chromecast_max_mean_down_std = df_chromecast_max_mean_down["log_bytes_down"].std()

In [ ]:
# use the previous mean and std to calculate the gaussian distribution using the gaussian function
chromecast_max_median_up_gaussian = gaussian(
    df_chromecast_max_median_up["log_bytes_up"], chromecast_max_median_up_mean, chromecast_max_median_up_std
)

chromecast_max_median_down_gaussian = gaussian(
    df_chromecast_max_median_down["log_bytes_down"], chromecast_max_median_down_mean, chromecast_max_median_down_std
)

chromecast_max_mean_up_gaussian = gaussian(
    df_chromecast_max_mean_up["log_bytes_up"], chromecast_max_mean_up_mean, chromecast_max_mean_up_std
)

chromecast_max_mean_down_gaussian = gaussian(
    df_chromecast_max_mean_down["log_bytes_down"], chromecast_max_mean_down_mean, chromecast_max_mean_down_std
)

### Histogram with MLE

In [ ]:
for df, column, gaussian, mean, std in [
    (df_chromecast_max_median_up, "log_bytes_up", chromecast_max_median_up_gaussian, chromecast_max_median_up_mean, chromecast_max_median_up_std),
    (df_chromecast_max_median_down, "log_bytes_down", chromecast_max_median_down_gaussian, chromecast_max_median_down_mean, chromecast_max_median_down_std),
    (df_chromecast_max_mean_up, "log_bytes_up", chromecast_max_mean_up_gaussian, chromecast_max_mean_up_mean, chromecast_max_mean_up_std),
    (df_chromecast_max_mean_down, "log_bytes_down", chromecast_max_mean_down_gaussian, chromecast_max_mean_down_mean, chromecast_max_mean_down_std)
]:
    fig = plt.figure()
    chromecast_hist_bin = calculate_bin(df[column])
    plt.hist(df[column], bins=chromecast_hist_bin, color=("blue" if column=="log_bytes_up" else "red"), alpha=0.5, edgecolor="black")
    plt.plot(df[column], gaussian)
    plt.xlabel("Bytes")
    plt.ylabel("Frequency")
    plt.legend(["Gaussian", "Histogram"])
    plt.title(f"Smart TV Histogram of {column} for Hour " + str(df["hour"].iloc[0]))
    fig.savefig(
        f"../images/chromecast/fourth_section/histogram_mle/chromecast_histogram_mle_{column}_hour_{df['hour'].iloc[0]}.png",
        dpi=fig.dpi,
    )
    print(f"Hour {df['hour'].iloc[0]} - {column} - mean: {mean}, std: {std}")

## Smart TV

### Filter Dataset

In [ ]:
hour_of_max_median_up_smart_tv = df_smart_tv.groupby("hour")["bytes_up"].median().idxmax()
hour_of_max_mean_up_smart_tv = df_smart_tv.groupby("hour")["bytes_up"].mean().idxmax()
hour_of_max_median_down_smart_tv = df_smart_tv.groupby("hour")["bytes_down"].median().idxmax()
hour_of_max_mean_down_smart_tv = df_smart_tv.groupby("hour")["bytes_down"].mean().idxmax()

In [ ]:
print(
    f"""
        Hour of max median bytes up: {hour_of_max_median_up_smart_tv},
        Hour of max median bytes down: {hour_of_max_median_down_smart_tv},
        Hour of max mean bytes up: {hour_of_max_mean_up_smart_tv}, 
        Hour of max mean bytes down: {hour_of_max_mean_down_smart_tv}
    """
)

In [ ]:
df_smart_tv_max_median_up = df_smart_tv[
    df_smart_tv["hour"] == hour_of_max_median_up_smart_tv
][["device_id", "hour", "bytes_up", "log_bytes_up"]]

df_smart_tv_max_median_down = df_smart_tv[
    df_smart_tv["hour"] == hour_of_max_median_down_smart_tv
][["device_id", "hour", "bytes_down", "log_bytes_down"]]

df_smart_tv_max_mean_up = df_smart_tv[
    df_smart_tv["hour"] == hour_of_max_mean_up_smart_tv
][["device_id", "hour", "bytes_up", "log_bytes_up"]]

df_smart_tv_max_mean_down = df_smart_tv[
    df_smart_tv["hour"] == hour_of_max_mean_down_smart_tv
][["device_id", "hour", "bytes_down", "log_bytes_down"]]

### Histogram

In [ ]:
for df, column in [
    (df_smart_tv_max_median_up, "log_bytes_up"),
    (df_smart_tv_max_median_down, "log_bytes_down"),
    (df_smart_tv_max_mean_up, "log_bytes_up"),
    (df_smart_tv_max_mean_down, "log_bytes_down")
]:
    smart_tv_hist_bin = calculate_bin(df[column])
    fig = plt.figure()
    plt.hist(df[column], bins=smart_tv_hist_bin, color=("blue" if column=="log_bytes_up" else "red"), alpha=0.5, edgecolor="black")
    plt.xlabel("Bytes")
    plt.ylabel("Frequency")
    plt.title(f"Smart TV Histogram of {column} for Hour " + str(df["hour"].iloc[0]))
    fig.savefig(
        f"../images/smart_tv/fourth_section/histogram/smart_tv_histogram_{column}_hour_{df['hour'].iloc[0]}.png",
        dpi=fig.dpi,
    )

### MLE

#### Gamma

#### Gaussian

In [ ]:
smart_tv_max_median_up_mean = df_smart_tv_max_median_up["log_bytes_up"].mean()
smart_tv_max_median_up_std = df_smart_tv_max_median_up["log_bytes_up"].std()

smart_tv_max_median_down_mean = df_smart_tv_max_median_down["log_bytes_down"].mean()
smart_tv_max_median_down_std = df_smart_tv_max_median_down["log_bytes_down"].std()

smart_tv_max_mean_up_mean = df_smart_tv_max_mean_up["log_bytes_up"].mean()
smart_tv_max_mean_up_std = df_smart_tv_max_mean_up["log_bytes_up"].std()

smart_tv_max_mean_down_mean = df_smart_tv_max_mean_down["log_bytes_down"].mean()
smart_tv_max_mean_down_std = df_smart_tv_max_mean_down["log_bytes_down"].std()

In [ ]:
smart_tv_max_median_up_gaussian = gaussian(
    df_smart_tv_max_median_up["log_bytes_up"], smart_tv_max_median_up_mean, smart_tv_max_median_up_std
)

smart_tv_max_median_down_gaussian = gaussian(
    df_smart_tv_max_median_down["log_bytes_down"], smart_tv_max_median_down_mean, smart_tv_max_median_down_std
)

smart_tv_max_mean_up_gaussian = gaussian(
    df_smart_tv_max_mean_up["log_bytes_up"], smart_tv_max_mean_up_mean, smart_tv_max_mean_up_std
)

smart_tv_max_mean_down_gaussian = gaussian(
    df_smart_tv_max_mean_down["log_bytes_down"], smart_tv_max_mean_down_mean, smart_tv_max_mean_down_std
)

### Histogram with MLE

In [ ]:
for df, column, gaussian, mean, std in [
    (df_smart_tv_max_median_up, "log_bytes_up", smart_tv_max_median_up_gaussian, smart_tv_max_median_up_mean, smart_tv_max_median_up_std),
    (df_smart_tv_max_median_down, "log_bytes_down", smart_tv_max_median_down_gaussian, smart_tv_max_median_down_mean, smart_tv_max_median_down_std),
    (df_smart_tv_max_mean_up, "log_bytes_up", smart_tv_max_mean_up_gaussian, smart_tv_max_mean_up_mean, smart_tv_max_mean_up_std),
    (df_smart_tv_max_mean_down, "log_bytes_down", smart_tv_max_mean_down_gaussian, smart_tv_max_mean_down_mean, smart_tv_max_mean_down_std)
]:
    fig = plt.figure()
    smart_tv_hist_bin = calculate_bin(df[column])
    plt.hist(df[column], bins=smart_tv_hist_bin, color=("blue" if column=="log_bytes_up" else "red"), alpha=0.5, edgecolor="black")
    plt.plot(df[column], gaussian)
    plt.xlabel("Bytes")
    plt.ylabel("Frequency")
    plt.legend(["Gaussian", "Histogram"])
    plt.title(f"Smart TV Histogram of {column} for Hour " + str(df["hour"].iloc[0]))
    fig.savefig(
        f"../images/smart_tv/fourth_section/histogram_mle/smart_tv_histogram_mle_{column}_hour_{df['hour'].iloc[0]}.png",
        dpi=fig.dpi,
    )
    print(f"Hour {df['hour'].iloc[0]} - {column} - mean: {mean}, std: {std}")